In [1]:
from loguru import logger
import requests
import configparser
import pandas as pd
import json
from units import download_by_addr
# logger.add('./log/VideoListSpider.log')
# logger.debug('this is a debug')
# logger.debug('this is a debug message')
# logger.info('this is another debug message')
# logger.warning('this is another debug message')
# logger.error('this is another debug message')
# logger.info('this is another debug message')
# logger.success('this is success message!')
# logger.critical('this is critical message!')

```
1 获取关键词视频列表 存储了视频的基本信息 记录aweme_id:7085652796254571817 sec_uid: MS4wLjABAAAA7JLzlrIRTwwYrwyTRHh0AvFGEcsfbNXt7jsSboxSZiI 10000/20*0.01
2 根据单个视频信息里的sec_uid获取用户信息
3 根据 aweme_id:7085652796254571817 下载视频 10000*0.01
4 根据aweme_id存储20条评论 10000*0.01
5 视频存到单个文件中 命名为aweme_id 
```

```
Target: 关键词视频下载
Save: 存储用户信息,视频信息,描述信息

```

### 爬取

In [2]:
keyword="运动 vlog"
payload={}
headers = {
'User-Agent': 'Apifox/1.0.0 (https://www.apifox.cn)'
}
total_video_list=[]
failed_target=0

In [3]:
## 爬取页号
for page in range(1,2):
    url = f"http://ttt.258data.com/dy/search/video/app/v3?keyword={keyword}&page={page}&token=AJOM2HCBCZGMVQQA"
    response = requests.request("GET", url, headers=headers, data=payload)
    scrapy_vedio_list=response.json()
    if scrapy_vedio_list['code']==0:
        failed_target=0
        print('Success') ## 请求返回成功了
        total_video_list.extend(scrapy_vedio_list['data']['aweme_msg_list'])
    else: # 请求失败 返回页号
        for i in range(3):
            response = requests.request("GET", url, headers=headers, data=payload)
            scrapy_vedio_list=response.json()
            if scrapy_vedio_list['code']==0:
                failed_target=0
                print('Success') ## 请求返回成功了
                total_video_list.extend(scrapy_vedio_list['data']['aweme_msg_list'])
                continue
        if scrapy_vedio_list['code']!=0:
            failed_target+=1
            logger.debug(f"Page {page} is scrapy failed , output : {scrapy_vedio_list['msg']}")
            if failed_target==3:
                logger.error(f'Page {page} is scrapy breaking')
                break

## 存储爬取信息
info=pd.DataFrame(total_video_list).drop_duplicates(subset=['aweme_id'],keep='first')
info.to_excel('info/search_运动 vlog.xlsx',encoding='utf_8_sig',index=False)

2022-12-30 14:56:36.099 | DEBUG    | __main__:<module>:21 - Page 1 is scrapy failed , output : token:AJOM2HCBCZGMVQQA 验证失败


In [3]:
info=pd.read_excel('info/search_运动 vlog.xlsx')
info.head()

,aweme_id,collect_count,comment_count,create_time,desc,digg_count,download_count,duration,follower_count,gender,...,sec_uid,share_count,share_url,short_id,signature,uid,unique_id,user_image_url,vedio_img,video_tag
0,7075313280675269896,19,5,1647349746,健身Vlog｜所有的快乐瞬间都值得记录🧩#运动#健身#vlog日常,497,6,84543,82988,2,...,MS4wLjABAAAAilksHSOuVpyJDk6c9KwqqaKI2ZpNntSKgG...,22,https://www.iesdouyin.com/share/video/70753132...,3571743469,有温度的法学生|你的能量补给站\nVlog|好物分享｜自我提升|一些碎碎念\n北外法语系&人...,434996167064872,dyxx76u0h94q,https://p6.douyinpic.com/aweme-avatar/tos-cn-a...,https://p26-sign.douyinpic.com/tos-cn-p-0015/e...,"[{'level': 1, 'tag_id': 2029, 'tag_name': '随拍'..."
1,7176212962951105849,73,79,1670842288,当你有个练体育的女儿… 就会时不时被拉去一起锻炼 #运动vlog日常 #运动女孩 #跑步,979,11,62167,147022,2,...,MS4wLjABAAAA5rKrYnkI-XaIHj74SMq8K4v97SSyIYgC-x...,62,https://www.iesdouyin.com/share/video/71762129...,98475951,🧣🍠同名\n🐷 @小🐑睡不醒,72413122800,NaN,https://p26.douyinpic.com/aweme-avatar/tos-cn-...,https://p6-sign.douyinpic.com/tos-cn-p-0015/43...,"[{'level': 1, 'tag_id': 2004, 'tag_name': '健身'..."
2,7127210145498582272,70,213,1659603627,运动打卡～又是运动量爆表的一天～#vlog日常,3572,14,103373,218321,2,...,MS4wLjABAAAAa6HudKVgrvxwZUaNg44fDaYsaCzQuZ9bRs...,60,https://www.iesdouyin.com/share/video/71272101...,94443830388,憨憨流水账VLOG📷\n@贺宝贺宝儿 \n🈴商务: starrank03\n电商v：3310...,4301706266946637,1070921p,https://p6.douyinpic.com/aweme-avatar/tos-cn-a...,https://p3-sign.douyinpic.com/tos-cn-p-0015/c0...,"[{'level': 1, 'tag_id': 2029, 'tag_name': '随拍'..."
3,7149119924470910222,406,278,1664703035,运动vlog｜看看我俩在健身房做了哪些努力～#运动#日常vlog #罗曼电动牙刷,16719,7,146560,1997840,2,...,MS4wLjABAAAAEHOngzwEpo9en_Gnh0r8t1hkkCY0IOiNwx...,551,https://www.iesdouyin.com/share/video/71491199...,2366212072,大家好我是宵夜札札\n一个不入流的伪分享博主\n🧣: 宵夜大哥 🍠: 宵夜大哥\n合作v：s...,2774774354809566,dywdnzg6pbaj,https://p26.douyinpic.com/aweme-avatar/tos-cn-...,https://p26-sign.douyinpic.com/tos-cn-i-dy/5ab...,"[{'level': 1, 'tag_id': 2026, 'tag_name': '科技'..."
4,7176236508649721091,12,21,1670847771,当我开始每天晨跑……这一周发生了什么……#运动 #好身材练起来 #自律 #跑步 #vlog,312,0,311680,71011,2,...,MS4wLjABAAAAdDQ0YiWStRFZ6cOwyECaFHuBvxGOJYPXk1...,13,https://www.iesdouyin.com/share/video/71762365...,2299906383,可能是全网穿男装最好看的女博主\n不止喜欢穿搭，还热爱生活！ 超会买！\n小Red书/🧣同名,109275116520,Darrrrcy,https://p6.douyinpic.com/aweme-avatar/tos-cn-a...,https://p3-sign.douyinpic.com/tos-cn-p-0015/og...,"[{'level': 1, 'tag_id': 2029, 'tag_name': '随拍'..."


In [4]:
info.apply(lambda x:download_by_addr('./video/'+str(x.aweme_id),x.play_addr),axis=1)

./video/7127210145498582272.mp4: 18.1MB [00:02, 6.22MB/s]                            
./video/7176236508649721091.mp4: 78.4MB [00:12, 2.15MB/s]                              
./video/7074121926976736547.mp4: 10.1MB [00:02, 4.29MB/s]                            
./video/7099683846739758366.mp4: 17.7MB [00:01, 10.2MB/s]                              
./video/7031118368933547275.mp4: 5.92MB [00:00, 9.50MB/s]                            
./video/7111241332273843496.mp4: 37.5MB [00:01, 20.9MB/s]                            
./video/7166918057313520908.mp4: 16.7MB [00:01, 12.8MB/s]                            


0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
9    None
dtype: object